## Enhancing road safety on Queensland roads

Based on historical data, we could predict the most dangerous roads for accidents based on road quality and other features that affect the probability of accidents.

The following questions came up while ideating over the safety theme at the hackathon:
* Given a location and date, what’s the expected number of accidents? 
* Which of the Brisbane Bike Week locations will be a hotspot for accidents?
* Which locations are outliers in the historical data? Or, are there any trends?
* Where does the road quality cause more accidents? Can we predict areas where road quality is a major factor in accidents?



In [24]:
%matplotlib inline
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import os
from sklearn.utils import shuffle

In [2]:
data_path = r'/Users/alexandraszenti/Documents/GitHub Projects/ARRBHack-2018/_data'

In [3]:
df = pd.read_csv(os.path.join(data_path,'road_crash_locations.csv'))

In [4]:
df.head()

,Crash_Ref_Number,Crash_Severity,Crash_Year,Crash_Month,Crash_Day_Of_Week,Crash_Hour,Crash_Nature,Crash_Type,Crash_Longitude_GDA94,Crash_Latitude_GDA94,...,Count_Casualty_MedicallyTreated,Count_Casualty_MinorInjury,Count_Casualty_Total,Count_Unit_Car,Count_Unit_Motorcycle_Moped,Count_Unit_Truck,Count_Unit_Bus,Count_Unit_Bicycle,Count_Unit_Pedestrian,Count_Unit_Other
0,1,Property damage only,2001,January,Monday,22,Hit object,Single Vehicle,153.028793,-27.449659,...,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2,Property damage only,2001,January,Monday,2,Overturned,Single Vehicle,152.966657,-27.049444,...,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Medical treatment,2001,January,Monday,14,Rear-end,Multi-Vehicle,152.821788,-27.573498,...,1,0,1,2.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Minor injury,2001,January,Monday,0,Hit object,Single Vehicle,152.974398,-27.225505,...,0,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Medical treatment,2001,January,Monday,15,Angle,Multi-Vehicle,153.002385,-27.248969,...,1,0,1,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
features =[]
for col in df.columns:
    features.append(col)
print(df.co)

['Crash_Ref_Number', 'Crash_Severity', 'Crash_Year', 'Crash_Month', 'Crash_Day_Of_Week', 'Crash_Hour', 'Crash_Nature', 'Crash_Type', 'Crash_Longitude_GDA94', 'Crash_Latitude_GDA94', 'Crash_Street', 'Crash_Street_Intersecting', 'Loc_Suburb', 'Loc_Local_Government_Area', 'Loc_Post_Code', 'Loc_Police_Division', 'Loc_Police_District', 'Loc_Police_Region', 'Loc_Queensland_Transport_Region', 'Loc_Main_Roads_Region', 'Loc_ABS_Statistical_Area_2', 'Loc_ABS_Statistical_Area_3', 'Loc_ABS_Statistical_Area_4', 'Loc_ABS_Remoteness', 'Loc_State_Electorate', 'Loc_Federal_Electorate', 'Crash_Controlling_Authority', 'Crash_Roadway_Feature', 'Crash_Traffic_Control', 'Crash_Speed_Limit', 'Crash_Road_Surface_Condition', 'Crash_Atmospheric_Condition', 'Crash_Lighting_Condition', 'Crash_Road_Horiz_Align', 'Crash_Road_Vert_Align', 'Crash_DCA_Code', 'Crash_DCA_Description', 'Crash_DCA_Group_Description', 'Count_Casualty_Fatality', 'Count_Casualty_Hospitalised', 'Count_Casualty_MedicallyTreated', 'Count_Casual

## Understanding correlation between features

To clean the data and know where to start exploring, we need a better understading of what features are more likely to correlate with FSI accidents and accidents involving cyclists or pedestrians. We first need to see the categorical data representations and convert them to numerical to allow to calculate correlation.

In [6]:
df.shape

(306558, 50)

In [7]:
df['Crash_Severity'].value_counts()

Medical treatment       90561
Property damage only    87448
Hospitalisation         83039
Minor injury            41088
Fatal                    4422
Name: Crash_Severity, dtype: int64

In [8]:
df['Crash_Day_Of_Week'].value_counts()

Friday       51834
Thursday     46730
Wednesday    44827
Saturday     43766
Tuesday      43136
Monday       41235
Sunday       35030
Name: Crash_Day_Of_Week, dtype: int64

In [9]:
df['Loc_ABS_Remoteness'].value_counts()

Major cities      182963
Inner regional     64709
Outer regional     47015
Remote              6440
Very remote         4707
Unknown              724
Name: Loc_ABS_Remoteness, dtype: int64

In [10]:
df['Crash_Road_Surface_Condition'].value_counts()

Sealed - Dry      249775
Sealed - Wet       48006
Unsealed - Dry      7449
Unsealed - Wet      1010
Unknown              318
Name: Crash_Road_Surface_Condition, dtype: int64

In [11]:
df['Count_Unit_Pedestrian'].value_counts()

0.0     293897
1.0      12200
2.0        386
3.0         48
4.0          7
5.0          2
20.0         1
11.0         1
Name: Count_Unit_Pedestrian, dtype: int64

In [12]:
df['Crash_Atmospheric_Condition'].value_counts()

Clear         266754
Raining        37447
Fog             1556
Smoke/Dust       582
Unknown          219
Name: Crash_Atmospheric_Condition, dtype: int64

In [13]:
df['Crash_Lighting_Condition'].value_counts()

Daylight                  211803
Darkness - Lighted         53200
Darkness - Not lighted     23586
Dawn/Dusk                  17589
Unknown                      380
Name: Crash_Lighting_Condition, dtype: int64

In [14]:
df['Crash_Month'].value_counts()

March        27358
August       27052
July         26578
May          26573
October      25863
June         25668
April        25657
November     25412
September    24786
December     24533
February     24071
January      23007
Name: Crash_Month, dtype: int64

In [15]:
 df['Crash_Roadway_Feature'].value_counts()

No Roadway Feature              163440
Intersection - T-Junction        63561
Intersection - Cross             48333
Intersection - Roundabout        13952
Intersection - Interchange        8269
Bridge/Causeway                   4796
Median Opening                    1588
Merge Lane                         685
Intersection - Multiple Road       553
Railway Crossing                   531
Forestry/National Park Road        383
Intersection - Y-Junction          218
Bikeway                            105
Other                               88
Intersection - 5+ way               55
Miscellaneous                        1
Name: Crash_Roadway_Feature, dtype: int64

In [16]:
df['Crash_DCA_Group_Description'].value_counts()

Rear-end                                    70683
Intersection from adjacent approaches       45595
Off carriageway on straight hit object      30939
Opposing vehicles turning                   27925
Other                                       25713
Off carriageway on curve hit object         21866
Vehicle leaving driveway                    11158
Pedestrian                                  10506
Head-on                                     10074
Lane changes                                 9426
Hit parked vehicle                           6930
Parallel lanes turning                       6928
Out of control on straight                   6519
Out of control on curve                      6110
Off carriageway on straight                  5714
Off carriageway on curve                     3616
Hit animal                                   2962
Overtaking same direction                    1705
U-turn                                       1682
Hit permanent obstruction on carriageway      260


### Cleaning the data to keep relevant features

Given the columns availabe, we are going to only keep information for the following:

['Crash_Ref_Number', 'Crash_Severity', 'Crash_Month', 'Crash_Day_Of_Week', 'Crash_Hour', 'Crash_Type', 'Crash_Longitude_GDA94', 'Crash_Latitude_GDA94','Loc_Suburb', 'Loc_ABS_Remoteness', 'Crash_Roadway_Feature', 'Crash_Traffic_Control', 'Crash_Speed_Limit', 'Crash_Road_Surface_Condition', 'Crash_Atmospheric_Condition', 'Crash_Lighting_Condition', 'Crash_Road_Horiz_Align', 'Crash_Road_Vert_Align', 'Count_Casualty_Fatality', 'Count_Casualty_Hospitalised', 'Count_Casualty_MedicallyTreated', 'Count_Casualty_MinorInjury', 'Count_Unit_Car', 'Count_Unit_Motorcycle_Moped', 'Count_Unit_Truck', 'Count_Unit_Bus', 'Count_Unit_Bicycle', 'Count_Unit_Pedestrian', 'Count_Unit_Other']

We will also need to remove those items where any of the values above is unknown.



In [17]:
df.loc[0, ['Crash_Ref_Number', 'Crash_Severity']]

Crash_Ref_Number                       1
Crash_Severity      Property damage only
Name: 0, dtype: object

In [18]:
df.shape

(306558, 50)

In [19]:
df[df.index < 200].to_csv(os.path.join(data_path,'cleaned_crash_locations_100.csv'),
                          index=False,
                         columns=['Crash_Severity', 'Crash_Month', 'Crash_Day_Of_Week', 'Crash_Hour','Loc_Suburb', 'Loc_ABS_Remoteness', 'Crash_Roadway_Feature', 'Crash_Traffic_Control', 'Crash_Speed_Limit', 'Crash_Road_Surface_Condition', 'Crash_Atmospheric_Condition', 'Crash_Lighting_Condition', 'Crash_Road_Horiz_Align', 'Crash_Road_Vert_Align', 'Count_Casualty_Fatality', 'Count_Casualty_Hospitalised', 'Count_Casualty_MedicallyTreated', 'Count_Casualty_MinorInjury', 'Count_Unit_Car', 'Count_Unit_Motorcycle_Moped', 'Count_Unit_Truck', 'Count_Unit_Bus', 'Count_Unit_Bicycle', 'Count_Unit_Pedestrian', 'Count_Unit_Other'])

In [20]:
df.to_csv(os.path.join(data_path,'cleaned_crash_locations_all.csv'),
                          index=False,
                         columns=['Crash_Severity', 'Crash_Month', 'Crash_Day_Of_Week', 'Crash_Hour','Loc_Suburb', 'Loc_ABS_Remoteness', 'Crash_Roadway_Feature', 'Crash_Traffic_Control', 'Crash_Speed_Limit', 'Crash_Road_Surface_Condition', 'Crash_Atmospheric_Condition', 'Crash_Lighting_Condition', 'Crash_Road_Horiz_Align', 'Crash_Road_Vert_Align', 'Count_Casualty_Fatality', 'Count_Casualty_Hospitalised', 'Count_Casualty_MedicallyTreated', 'Count_Casualty_MinorInjury', 'Count_Unit_Car', 'Count_Unit_Motorcycle_Moped', 'Count_Unit_Truck', 'Count_Unit_Bus', 'Count_Unit_Bicycle', 'Count_Unit_Pedestrian', 'Count_Unit_Other'])

In [21]:
df.to_csv(os.path.join(data_path,'cleaned_crash_locations_cyclists.csv'),
                          index=False,
                         columns=['Crash_Severity', 'Crash_Month', 'Crash_Day_Of_Week', 'Crash_Hour','Loc_Suburb', 'Loc_ABS_Remoteness', 'Crash_Longitude_GDA94', 'Crash_Latitude_GDA94','Crash_Roadway_Feature', 'Crash_Speed_Limit', 'Crash_Road_Surface_Condition', 'Crash_Atmospheric_Condition', 'Crash_Lighting_Condition', 'Crash_Road_Horiz_Align', 'Crash_Road_Vert_Align', 'Count_Casualty_Fatality', 'Count_Casualty_Hospitalised', 'Count_Casualty_MedicallyTreated', 'Count_Casualty_MinorInjury', 'Count_Unit_Bicycle'])

In [22]:
df.to_csv(os.path.join(data_path,'cleaned_crash_locations_ped_and_cyc.csv'),
                          index=False,
                         columns=['Crash_Severity', 'Crash_Month', 'Crash_Day_Of_Week', 'Crash_Hour','Loc_Suburb', 'Loc_ABS_Remoteness', 'Crash_Longitude_GDA94', 'Crash_Latitude_GDA94','Crash_Roadway_Feature', 'Crash_Speed_Limit', 'Crash_Road_Surface_Condition', 'Crash_Atmospheric_Condition', 'Crash_Lighting_Condition', 'Crash_Road_Horiz_Align', 'Crash_Road_Vert_Align', 'Count_Casualty_Fatality', 'Count_Casualty_Hospitalised', 'Count_Casualty_MedicallyTreated', 'Count_Casualty_MinorInjury', 'Count_Unit_Pedestrian','Count_Unit_Bicycle'])